In [0]:
# Parameters
sql_server_name = "xxxxxxx-beverage-sql-server.database.windows.net"
sql_database = "xxxxxxx-beverage-sql" 
sql_user = "xxxxxxx_teste"
sql_password = "xxxxxxxxxxxx"

# JDBC connection
jdbc_url = f"jdbc:sqlserver://{sql_server_name}:1433;database={sql_database};encrypt=true;trustServerCertificate=false;hostNameInCertificate=*.database.windows.net;loginTimeout=30;"

connection_properties = {
    "user": sql_user,
    "password": sql_password, 
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}

In [0]:
# Temp_view to sql_query
fact_sale_df = spark.read.jdbc(jdbc_url, "dbo.sql_fact_sale", properties=connection_properties)
channel_group_df = spark.read.jdbc(jdbc_url, "dbo.sql_dim_channel_group", properties=connection_properties)
month_df = spark.read.jdbc(jdbc_url, "dbo.sql_dim_month", properties=connection_properties)
flavor_df = spark.read.jdbc(jdbc_url, "dbo.sql_dim_flavor", properties=connection_properties)

fact_sale_df.createOrReplaceTempView("sql_fact_sale")
channel_group_df.createOrReplaceTempView("sql_dim_channel_group")
month_df.createOrReplaceTempView("sql_dim_month")
flavor_df.createOrReplaceTempView("sql_dim_flavor")

In [0]:
# Gold Layer path variables
RESULT_REGION_TRADE_GROUP = "agg_sales_by_region_trade_group"
RESULT_BRAND_MONTH = "agg_sales_by_brand_month"
RESULT_LOWEST_BRAND = "agg_lowest_brand_by_region"


In [0]:
# Top 3 Trade Groups for each Region in sales ($ Volume)
view_region_trade_group = spark.sql(f"""
WITH ranked_data AS (
    SELECT 
        fs.region_desc AS region,
        dcg.trade_group_desc AS trade_group,
        SUM(fs.sales_volume) AS total_volume,
        ROW_NUMBER() OVER(PARTITION BY fs.region_desc ORDER BY SUM(fs.sales_volume) DESC) AS rank
    FROM sql_fact_sale fs
    JOIN sql_dim_channel_group dcg ON fs.channel_id = dcg.channel_id
    GROUP BY fs.region_desc, dcg.trade_group_desc
)
SELECT region, trade_group, total_volume
FROM ranked_data 
WHERE rank <= 3
ORDER BY region, total_volume DESC
""")

In [0]:
# Sales each Brand per month
view_brand_month = spark.sql(f"""
SELECT 
    df.flavor_description AS brand,
    fs.year,
    dm.month_name AS month,
    SUM(fs.sales_volume) AS total_sales
FROM sql_fact_sale fs
    JOIN sql_dim_month dm ON fs.month_id = dm.month_id
    JOIN sql_dim_flavor df ON fs.flavor_id = df.flavor_id
GROUP BY df.flavor_description, fs.year, dm.month_name
ORDER BY year, month, total_sales DESC
""")

In [0]:
# Lowest brand in sales for each Region
view_lowest_brand = spark.sql(f"""
WITH ranked_brands AS (
    SELECT 
        fs.region_desc AS region,
        df.flavor_description AS brand,
        SUM(fs.sales_volume) AS total_sales,
        ROW_NUMBER() OVER(PARTITION BY fs.region_desc ORDER BY SUM(fs.sales_volume) ASC) AS rank
    FROM sql_fact_sale fs
        JOIN sql_dim_flavor df ON fs.flavor_id = df.flavor_id
    GROUP BY fs.region_desc, df.flavor_description
)
SELECT 
    region, 
    brand, 
    total_sales
FROM ranked_brands
WHERE rank = 1
ORDER BY region
""")

In [0]:
def write_to_sql(df, table_name):
    df.write \
        .format("jdbc") \
        .option("url", jdbc_url) \
        .option("dbtable", f"dbo.{table_name}") \
        .option("user", sql_user) \
        .option("password", sql_password) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .mode("overwrite") \
        .save()

In [0]:
write_to_sql(view_region_trade_group, RESULT_REGION_TRADE_GROUP)
write_to_sql(view_brand_month, RESULT_BRAND_MONTH)
write_to_sql(view_lowest_brand, RESULT_LOWEST_BRAND)